<a href="https://colab.research.google.com/github/arnisafazla/CS490/blob/main/CS_490_notes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
%cd /content/drive/MyDrive/CS490/CS490-Senior-Research-Project

/content/drive/MyDrive/CS490/CS490-Senior-Research-Project


In [3]:
! git pull

Already up to date.


In [3]:
!git config --global user.email "arnisa.fazla@ug.bilkent.edu.tr"
!git config --global user.name "arnisafazla"

In [14]:
! git commit -m 'debug train function'

On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
	modified:   __pycache__/dataset.cpython-37.pyc
	modified:   models/Base_WGAN.py
	modified:   models/__pycache__/Base_WGAN.cpython-37.pyc
	modified:   models/critic_models/__pycache__/LSTM_critic.cpython-37.pyc
	modified:   setup.ipynb

no changes added to commit


In [23]:
! touch .gitignore

In [27]:
! git add .gitignore

In [25]:
! echo 'models/__pycache__' >> .gitignore

In [13]:
! git add models/
! git add tools.py

In [29]:
! git push origin main

Counting objects: 23, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (22/22), done.
Writing objects: 100% (23/23), 24.62 KiB | 128.00 KiB/s, done.
Total 23 (delta 8), reused 0 (delta 0)
remote: Resolving deltas: 100% (8/8), completed with 7 local objects.
To https://github.com/arnisafazla/CS490-Senior-Research-Project.git
   3398b38..a4ef9ac  main -> main


In [3]:
import os, sys
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from keras.utils.vis_utils import plot_model

# path to the cloned repo
main_dir = '/content/drive/MyDrive/CS490/CS490-Senior-Research-Project'  
# path to the dataset folder
dataset_dir = '/content/drive/MyDrive/CS490/dataset'
%cd {main_dir}
sys.path.insert(0,main_dir)
from dataset import Dataset
from models.critic_models.LSTM_critic import define_critic
from models.generator_models.LSTM_generator import define_generator
from models.Base_WGAN import Base_WGAN

/content/drive/MyDrive/CS490/CS490-Senior-Research-Project


In [5]:
%cd ..
!mkdir generator_models
%cd generator_models
! touch LSTM_generator.py

/content/drive/MyDrive/CS490/CS490-Senior-Research-Project/models
/content/drive/MyDrive/CS490/CS490-Senior-Research-Project/models/generator_models


In [4]:
dataset = Dataset(["anger", "joy", "fear", "neutral", "sadness", "pride"], path='/content/drive/MyDrive/CS490/dataset', step_size=10, frames=400)

anger


100%|██████████| 65/65 [00:10<00:00,  6.07it/s]


joy


100%|██████████| 29/29 [00:05<00:00,  5.65it/s]


fear


...

100%|██████████| 84/84 [00:11<00:00,  7.15it/s]


sadness


100%|██████████| 46/46 [00:07<00:00,  6.05it/s]


pride


100%|██████████| 24/24 [00:04<00:00,  5.39it/s]


In [5]:
Dataset.balance(dataset)

No. of samples in each class will be:  640


In [6]:
# save this as a separate file
config = {       # change hyperparams here
        'latent_dim' : 400,
        'in_shape' : (400, 69),
        'n_classes' : 6,
        'init_std' : 0.01,
        'critic_opt' : 'Adam', # RMS or Adam
        'critic_lr' : 0.005,
        'gen_opt' : 'Adam',
        'gen_lr' : 0.005,
        'wrong_labels' : True,    # feed in real data with wron labels as well
        'epochs' : 20,
        'n_batch' : 128,
        'n_critic' : 5,
        'dataset_balanced' : True,
        'critic_batch_norm' : True,
        'critic_dropout' : 0,
        'gp_weight' : 10.0      # for the gradient penalty used in critic
      }

In [7]:
critic = define_critic(config)
# plot_model(critic, show_shapes=True, show_layer_names=True)
generator = define_generator(config)
# plot_model(generator, show_shapes=True, show_layer_names=True)

{'latent_dim': 400, 'in_shape': (400, 69), 'n_classes': 6, 'init_std': 0.01, 'critic_opt': 'Adam', 'critic_lr': 0.005, 'gen_opt': 'Adam', 'gen_lr': 0.005, 'wrong_labels': True, 'epochs': 20, 'n_batch': 128, 'n_critic': 5, 'dataset_balanced': True, 'critic_batch_norm': True, 'critic_dropout': 0, 'gp_weight': 10.0}


In [8]:
gan = Base_WGAN('/content/drive/MyDrive/CS490/models', 'WGAN', None, critic, generator, config, dataset)

In [11]:
generator_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)
critic_optimizer = keras.optimizers.Adam(
    learning_rate=0.0002, beta_1=0.5, beta_2=0.9
)
# Define the loss functions for the critic,
# which should be (fake_loss - real_loss).
# We will add the gradient penalty later to this loss function.
def critic_loss(real, fake):
    real_loss = tf.reduce_mean(real)
    fake_loss = tf.reduce_mean(fake)
    return fake_loss - real_loss
# Define the loss functions for the generator.
def generator_loss(fake):
    return -tf.reduce_mean(fake)

# gan = Base_WGAN('/content/drive/MyDrive/CS490/models', 'WGAN', None, critic, generator, config, dataset)

# Compile the WGAN model.
gan.compile(
    c_optimizer=critic_optimizer,
    g_optimizer=generator_optimizer,
    g_loss_fn=generator_loss,
    c_loss_fn=critic_loss,
)

# Start training the model.
# wgan.fit(train_images, batch_size=BATCH_SIZE, epochs=epochs, callbacks=[cbk])

In [12]:
gan.train('/content/drive/MyDrive/CS490/logs')

AttributeError: ignored